# DTGraph Introduction

This notebook consists of simple examples of usage of the library.

## Part 1: Establishing a connection

In [ ]:
from dtgraph import Neo4jGraph, Rule

In the following, we will connect to a [Neo4j Sandbox](https://sandbox.neo4j.com/).
After having created your instance, you can retrieve the *ip address* and the *password* (e.g., in my case, those are  `54.159.136.111` and `analyzer-information-tails`).

In [2]:
hostname = "54.159.136.111"
password = "analyzer-information-tails"

In [3]:
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, "neo4j", username="neo4j", password=password)

You can check that it has some content if you specified the Movie dataset:

In [4]:
graph.output_all_nodes()

Info: There is currently 273 node(s) in the database.


## Part II: (Optional) Loading a curated scenario

If your database is empty or you don't have it loaded with the *Movie dataset*, you can import it using the following command:

In [5]:
from dtgraph.scenarios.movies import Movies
Movies.load(graph)

Flushed database: Deleted 273 nodes, deleted 1021 relationships, completed after 8 ms.
Load scenario: Added 171 labels, created 171 nodes, set 564 properties, created 253 relationships, completed after 3 ms.


## Part III: Defining and executing a rule

Then you can write your own transformation rule:

In [6]:
my_query = Rule('''
        MATCH (n:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(o:Person)
        => 
        (x = (n) : Actor {
            name = n.name,
            born = n.born
        })-[(m) : COLLEAGUE {
            movie = m.title
        }]->(y = (o) : Actor {
            name = o.name,
            born = o.born
        })
''')

And execute it:

In [7]:
my_query.apply_on(graph)

Rule: Added 204 labels, created 102 nodes, set 4710 properties, created 768 relationships, completed after 1054 ms.


Yay, 102 nodes and 204 labels have been created! You can check with the following query on your Neo4j browser the result (alongside the initial dataset, for now):
```
MATCH (n)
RETURN n
```